In [2]:
import torch as tr
import numpy as np
from CSW import *

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# match human data
- to match human i need to prop one sentence at a time. otherwise the network will do well on later sentences (because after observing one transition the next are determined)
- backprop for each sample, flush at end of story

In [33]:
task = CSWTask(graph_pr=1.0)

In [42]:
path = task.sample_path()
task.graph

{0: {1: 0.5, 2: 0.5},
 1: {3: 1.0, 4: 0.0},
 2: {3: 0.0, 4: 1.0},
 3: {5: 1.0, 6: 0.0},
 4: {5: 0.0, 6: 1.0},
 5: {7: 1.0, 8: 0.0},
 6: {7: 0.0, 8: 1.0},
 7: {9: 1.0},
 8: {9: 1.0}}

In [47]:

X,Y = task.dataset_onestory_with_marker(path=path,
                                        filler_id=10,
                                        depth=1)
X

tensor([[10],
        [ 0],
        [ 2],
        [ 4],
        [ 6]])

In [48]:
for o in range(len(X)):
  print('x=',X[o],'y=',Y[o])

x= tensor([10]) y= tensor([0])
x= tensor([0]) y= tensor([2])
x= tensor([2]) y= tensor([4])
x= tensor([4]) y= tensor([6])
x= tensor([6]) y= tensor([8])


In [16]:
task.sample_path()

array([0, 2, 4, 6, 8, 9])

In [3]:
# task
pr = 1.0
taskL = [CSWTask(pr),CSWTask(1-pr)]

eval_pathL = [
  [10,0,1,4,5],       
  [11,0,1,3,5],
  ]

xeval = taskL[0].format_Xeval(eval_pathL)
xeval = tr.tensor(xeval)

In [3]:
## params
stsize = 40
learn_rate = 0.005
neps = 200
block_len = 1
## train setup
softmax = lambda ulog: tr.softmax(ulog,-1)
lossop = tr.nn.CrossEntropyLoss()


## loop over seeds
# for seed in range(10):
#   lossL = []
  # init net
  net = CSWNet(stsize,seed)
  optiop = tr.optim.Adam(net.parameters(), lr=learn_rate)
  model_name='ysm-blocklen%i-seed%i'%(block_len,seed)
  # eval array
  tdim,sm_dim=5,12
  ysm = -np.ones([2,neps,tdim,sm_dim])
  ## train loop
  task_int = 0
  for ep in range(neps):
    if ep == 160:
      block_len = 1
    # select graph
    if ep%block_len==0:
      task_int = (task_int+1)%2
      task = taskL[task_int]
      filler_id = 10+task_int 
    # forward prop  
    path = task.sample_path()
    xtrain,ytrain = task.dataset_onestory_with_marker(path=path,filler_id=filler_id,depth=1)
    yh = net(xtrain) # (time,smunits)
    # eval
    for idx,xev in enumerate(xeval):
      ysm_t = softmax(net(xev)).detach().numpy()
      ysm[idx,ep] = ysm_t
    # backprop
    loss = 0
    for tstep in range(len(xtrain)):
      loss = lossop(yh[tstep].unsqueeze(0),ytrain[tstep])
      optiop.zero_grad()
      loss.backward(retain_graph=True)
      optiop.step()
    lossL.append(loss)
#     if verb and ep%(neps/5)==0:
#       print(ep/neps,loss.detach().numpy())
  fpath = 'eval_data/human_match/'+model_name 
#   np.save(fpath,ysm)

seed 0
seed 1
seed 2
seed 3
seed 4
seed 5
seed 6
seed 7
seed 8
seed 9


In [5]:
plt.plot(lossL)